# Python - the missing bits.
# Practical metaprogramming applications.

#### Michał Korzycki - 

> ### [Metaclasses] are deeper magic than 99% of users should ever worry about. 
> ### If you wonder whether you need them, you don’t 
> ### (the people who actually need them know with certainty that they need them, and don’t need an explanation about why).
>
> #### <div style="text-align: right">-- Tim Peters, Inventor of the timsort algorithm and prolific Python contributor</span>

> ### I call it "the advanced beginner metaclass trap".
> ### <div style="text-align: right">... random comment on the internet</span>

### 99% of the time actually you only need __subclass_init__.
 
####  `__subclass_init__`  was only added to Python 3.6 in 2015 via PEP 487.

# Agenda

- ### Metaclasses refresher
- ### *"Real"* Singletons are not evil
- ### *Final* classes
- ### Java-like Annotations are also Meta
- ### Meta-models for SQLAlchemy
- ### General guidelines

# Metaclasses refresher

In [ ]:
When you create an instance of a class, Python first calls the __new__() method to create the object and then calls the __init__() method to initialize the object’s attributes.

`object.__new__(cls[, ...])`
Called to create a new instance of `class cls`

`object.__init__(self[, ...])`
Called after the instance has been created (by __new__())

`__new__()` is intended mainly to allow subclasses of immutable types <...> to customize instance creation. It is also commonly overridden in custom metaclasses in order to customize class creation.

In [ ]:
__prepare__ is invoked as a function before the evaluation of the class body. 

# <span style="color: cyan">*"Real"*</span>&nbsp; Singletons are not evil

### It is all about <span style="color: cyan">*subclassing*</span>

> ## Singletons aren't *polymorphic*.
> ### <div style="text-align: right">Wiki Wiki Web (wiki.c2.com)</span>Singletons aren't *polymorphic*.

---

> ## When you have singletons in your code, it makes it hard to test and debug.
> ### <div style="text-align: right">David Litvak</span>


GoF book - about subclassing

```python
class ParentSingletonMeta(type):
    def __call__(cls, *args, **kwargs):
        parent = cls.__bases__[-1]

        if parent not in _singleton_instances:
            instance = super().__call__(*args, **kwargs)
            _singleton_instances[parent] = instance
        return _singleton_instances[parent]

class ABCParentSingletonMeta(ParentSingletonMeta, abc.ABCMeta):
    pass

```

```python
class WebPushProvider(metaclass=ParentSingletonMeta):
    @classmethod
    def get_webpush_notification_data(cls, title, body, delivery_id):
...
...
...
    @classmethod
    @abstractmethod
    def web_push(cls, domain, email, data, db=None):
        pass
```

```python
class DummyWebPushProvider(WebPushProvider):
    data = None

    @classmethod
    def web_push(cls, domain, email, data, db=None):
        cls.data = domain, email, data

                
class StandardWebPushProvider(WebPushProvider):
    @classmethod
    def web_push(cls, domain, email, data, db=None):
        subscriptions = get_subscriptions(domain, email, db)
...
```

# <span style="color: cyan">*Final*</span> &nbsp;classes

```python
class EnumMeta(type):
    """
    Metaclass for Enum
    """
    @classmethod
    def __prepare__(metacls, cls, bases, **kwds):
        ...
        first_enum = bases[-1]
        if first_enum._member_names_:
            raise TypeError("Cannot extend enumerations")
        ...
```

# Java-like annotations are also <span style="color: cyan">*Meta*</span>

### Use `__docs__` to carry information

```python
@router.get("/messages/{slug}", response_class=ORJSONResponse, response_model=List[Message])
@authorize('CONFIRMED')
async def get_messages(slug: str, Authorize: AuthJWT = Depends(), db: Session = Depends(get_db)):
    return crud.messages(slug, access_token_from_authorize(slug, Authorize), db)
```

```python
def authorize(permission=None, refresh=False):
    def _authorize(_func):
        @wraps(_func)
        async def wrapper(*args, **kwargs):
            try:
                Authorize = kwargs.get('Authorize')          
...
...
...
        wrapper.__docs__ = "Authorized"
        return wrapper
```

```python
@pytest.mark.parametrize("router_fixture", [
    ('agent-panel', api.agent_panel.router, ['/crmquestions-hello']),
...
...
...
], indirect=True)
def test_secops(router_fixture):
    path, router, whitelist = router_fixture
...
...
```

```python
def test_secops(router_fixture):
    path, router, whitelist = router_fixture
...
    for route in routes:
        assert   ((route.path, route.methods, route.endpoint.__docs__) 
               == (route.path, route.methods, 'Authorized'))
        
```

# General guidelines

- Be cautious about using Metaclasses
    - 99% of the time you just need `__init_subclass__`
- Do not mix metaclasses - that is not what the creators of libraries intended
- Read *PEP 487 – Simpler customisation of class creation*

In [ ]:
## Thank You!

## Like and subscribe: https://www.youtube.com/@BitsofData/streams